In [1]:
import sys
cpath = !pwd
sys.path.append('/usr/src/app/algorithms/')
sys.path.append('/usr/src/app/')

In [2]:
from lionforests import LionForests
from algorithms.simpleSurrogate import GlobalSurrogateTree, LocalSurrogateTree
from algorithms.DefragTrees.defragTrees import DefragModel
from CHIRPS.structures import data_container
import CHIRPS.routines as rt
import CHIRPS.structures as strcts
from algorithms.anchor.anchor_tabular import AnchorTabularExplainer

from scipy import sparse

from datasets.dataset import Dataset
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd 
import numpy as np
np.seterr(invalid='ignore')
import warnings
warnings.filterwarnings("ignore")
import time

from sklearn.model_selection import train_test_split, LeaveOneOut
from sklearn.metrics import f1_score, precision_score

segment = Dataset()

In [3]:
X, y, feature_names, class_names = segment.load_segment()

In [4]:
y = [int(i) for i in y]
y = np.array(y)

In [11]:
def measure(X_train, X_test, y_train, y_test, feature_names, class_names, random_state=10):
    parameters = [{
        'max_depth': [10],
        'max_features': [0.75],
        'bootstrap': [False],
        'min_samples_leaf' : [1],
        'n_estimators': [500]
    }]
    lf = LionForests(None, False, None, feature_names, class_names)
    lf.fit(X_train, y_train, params=parameters)    
    train = lf.utilizer.transform(X_train)
    test = lf.utilizer.transform(X_test)

    predictions = lf.model.predict(train)
    test_predictions = lf.model.predict(test)
    
    def techniques(model, train, y_train, predictions, test, feature_names, class_names, lf, task, random_state=10):

        #BaselineTechnique ==============================================================================
        gt = GlobalSurrogateTree(train, predictions, feature_names, task, random_state)
        lt = LocalSurrogateTree(train, predictions, feature_names, task, 150, random_state)

        #DefragTechnique ================================================================================
        Kmax = 10
        splitter = DefragModel.parseSLtrees(model) # parse sklearn tree ensembles into the array of (feature index, threshold)
        mdl = DefragModel(modeltype=task, maxitr=100, qitr=0, tol=1e-6, restart=20, verbose=0, seed=random_state)
        mdl.fit(train, predictions, splitter, Kmax, fittype='FAB', featurename=feature_names)
        def def_cov(instances):
            ts = time.time()
            score, cover, coll = mdl.evaluate(instances[1:],lf.model.predict(instances[1:]))
            def_predictions = mdl.predict(instances[1:])
            length, nodes = mdl.find_rule_length(instances[0])
            max_len = len(length)
            comp = {1:'>',0:'<='}
            rules = {}
            for f in feature_names:
                rules[f] = []
            counter = 0
            for rule, node in list(zip(mdl.rule_,nodes)):
                for conj in range(len(node)):
                    if node[conj] and counter<=max_len-1:
                        rules[feature_names[int(rule[conj][0]-1)]].append([comp[int(rule[conj][1])],rule[conj][2]])
                        counter += 1
            new_rules = {}
            for k, v in rules.items():
                if len(v) == 1:
                    new_rules[k] = v
                else:
                    mmin = None
                    mmax = None
                    for value in v:
                        if value[0]=='<=':
                            if mmin is None or mmin > value[1]:
                                mmin = value[1]
                        if value[0]=='>':
                            if mmax is None or mmax < value[1]:
                                mmax = value[1]
                    if mmin is not None:
                        new_rules[k] = [['<=', mmin]]
                    if mmax is not None:
                        new_rules[k] = [['>', mmax]]
            te = time.time()
            #print(mdl)
            return length, cover, def_predictions, te-ts, new_rules

        #Anchors =======================================================================================
        explainer = AnchorTabularExplainer(class_names, feature_names, train)
        def anchors_method(instance):
            exp = explainer.explain_instance(instance, lf.model.predict, threshold=0.95, random_state=random_state)
            anchors_dict = {}
            for i in exp.names():
                terms = i.split(' ')
                if len(terms) == 3:
                    anchors_dict[terms[0]] = [[terms[1],float(terms[2])]]
                else:
                    anchors_dict[terms[2]] = [[terms[3],float(terms[4])],[terms[1],float(terms[0])]]
            return anchors_dict, 0

        #CHIRPS =======================================================================================
        project_dir = 'C:\\Users\\iamollas\\Downloads\\LionForests Journal\\algorithms\\CHIRPS'
        temp_y = np.array([int(i+1) for i in y_train])
        temp_frame = pd.DataFrame(np.hstack((train,temp_y.reshape(len(temp_y),1))),columns=feature_names+['class'])
        temp_frame['class']=temp_frame['class'].astype(int)
        mydata = data_container(
                data = temp_frame, class_col = 'class', var_names = feature_names,
                project_dir = project_dir, save_dir = 'segment', random_state=random_state)
        meta_data = mydata.get_meta()
        f_walker = strcts.classification_trees_walker(forest=model, meta_data=meta_data)
        f_walker.forest_walk(instances = test, labels = model.predict(test), forest_walk_async = False)

        explanations = strcts.CHIRPS_container(f_walker.path_detail,
                                        forest=model,
                                        sample_instances=sparse.csr_matrix(train), # any representative sample can be used
                                        sample_labels=predictions,
                                        meta_data=meta_data)
        chts = time.time()
        explanations.run_explanations(target_classes=model.predict(test), # we're explaining the prediction, not the true label!
                                explanation_async=False,
                                random_state=random_state,
                                which_trees='majority',
                                alpha_paths=0.0,
                                support_paths=0.1,
                                score_func=1,
                                precis_threshold=0.99,
                                disc_path_bins=4,
                                merging_bootstraps=20,
                                pruning_bootstraps=20,
                                delta=0.2,
                                weighting='kldiv')
        chte = (time.time() - chts)/len(test)
        def chirps_method(idx):
            chirps_dict = {}
            for i in explanations.explainers[idx].pruned_rule:
                if i[1]:
                    chirps_dict[i[0]] = [['<=',float(i[2])]]
                else:
                    chirps_dict[i[0]] = [['>',float(i[2])]]
            return chirps_dict, 0, chte

        #LionForests
        def lf_rule(instance):
            temp = lf.explain(instance, instance_random_state=random_state)[5]
            rule = {}
            for key,value in temp.items():
                rule[key] = [['<=',value[1]],['>',value[0]]]
            return rule

        return {'gs':gt.rule,'ls':lt.rule,'an':anchors_method,'lf':lf_rule, 'df': def_cov, 'ch':chirps_method}
    interpretation = techniques(lf.model, train, y_train, predictions, test, feature_names, class_names, lf, 'classification', random_state)
    def rule_cov(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]: #<=
                        return 0
                    if instance[k] <= rule[feature][1][1]:#THIS <=
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]: #<=
                        return 0
        return 1
    def rule_cov_LF(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]: #<=
                        return 0
                    if instance[k] < rule[feature][1][1]:#THIS <=
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]: #<=
                        return 0
        return 1

    gt = GlobalSurrogateTree(train,lf.model.predict(train),feature_names,'c')
    lt = LocalSurrogateTree(train,lf.model.predict(train),feature_names,'c',150)

    rule_generator = interpretation
    full_coverage = {'gs':0, 'ls':0, 'an':0, 'lf':0, 'df':0, 'ch': 0}
    rule_length = {'gs':0, 'ls':0, 'an':0, 'lf':0, 'df':0, 'ch': 0}
    f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
    time_response = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
    rules = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}

    x_train_temp = train
    x_test_temp = test

    y_train_temp = predictions
    y_test_temp = test_predictions
    
    x_train_temp_lf = lf.utilizer.inverse_transform(x_train_temp)
    x_test_temp_lf = lf.utilizer.inverse_transform(x_test_temp)    
    ktime = time.time()
    
    for test_ind in range(len(test)):
        if test_ind % 100 ==0:
            print(round(test_ind/(len(test))*100,2),'in:', time.time()-ktime)
            ktime = time.time()
        for name, method in rule_generator.items():
            if name == 'ch':
                rule, op, te = method(test_ind)
                coverage = 0
                precision = []
                co = 0
                for i in x_test_temp:
                    res = rule_cov(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1:
                        precision.append([y_test_temp[test_ind], y_test_temp[co]])
                    co = co + 1
                if len(precision) > 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_test_temp)
                rules['ch'].append(rule)
            elif name == 'df':
                #FIX RULE LENGTH!
                rule, cover, predictions, te, new_rules = method(np.concatenate((x_test_temp[test_ind:test_ind+1],x_test_temp)))
                f_precision[name].append(precision_score(predictions,y_test_temp,average='micro'))
                full_coverage[name] = full_coverage[name] + cover
                rules['df'].append(new_rules)
            elif name == 'lf':
                ts = time.time()
                rule = method(x_test_temp_lf[test_ind])
                te = time.time() - ts
                coverage = 0
                precision = []
                co = 0
                for i in x_test_temp_lf:
                    res = rule_cov_LF(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1:
                        precision.append([y_test_temp[test_ind], y_test_temp[co]])
                    co = co + 1
                if len(precision) > 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_test_temp_lf)
                rules['lf'].append(rule)
            else:
                ts = time.time()
                rule, prediction = method(x_test_temp[test_ind])
                te = time.time() - ts
                coverage = 0
                precision = []
                co = 0 
                for i in x_test_temp:
                    if name == 'anchors':
                        res = rule_cov(i, feature_names, rule)
                    else:    
                        res = rule_cov(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1 and name=='anchors':
                        precision.append([y_test_temp[test_ind], y_test_temp[co]])
                    elif res == 1:
                        precision.append([prediction, y_test_temp[co]])
                    co = co + 1
                if len(precision) > 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_test_temp)
                rules[name].append(rule)
            time_response[name].append(te)
            rule_length[name] = rule_length[name] + len(rule)
    return rule_generator, full_coverage, rule_length, f_precision, time_response, rules

In [18]:
total_results2 = []
test_size_2 = []
for rand in [7, 10, 77]:
    total_results = []
    test_size = []
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=7)
    results = measure(X_train, X_test, y_train, y_test, feature_names, class_names, random_state=rand)
    total_results.append(results)
    test_size.append(len(X_test))
    test_size_2.append(test_size)
    total_results2.append(total_results)

len self.path_detail
462
Working on CHIRPS for instance 0 of 462
as_chirps for batch_idx 0
start mining for batch_idx 0 with support = 0.1
reduced 305 patterns out of 650 by numeric redundancy
found 345 patterns from 495 trees for batch_idx 0
start score sort for batch_idx 0 (345) patterns
start merge rule for batch_idx 0 (345) patterns
[('hue-mean', True, 0.18032), ('rawred-mean', True, 0.00324), ('region-centroid-row', True, 0.50471)]
0.8717948717948718 0.03818181323109765 0.1402048986013501 0.22354841900491282
merge complete for batch_idx 0 (345) patterns
start get explainer for batch_idx 0
as_chirps for batch_idx 1
start mining for batch_idx 1 with support = 0.1
found 148 patterns from 499 trees for batch_idx 1
start score sort for batch_idx 1 (148) patterns
start merge rule for batch_idx 1 (148) patterns
[('hue-mean', False, 0.14151), ('rawred-mean', False, 0.18144), ('region-centroid-row', True, 0.61229), ('rawgreen-mean', False, 0.1629), ('intensity-mean', True, 0.58637)]
0.9574

reduced 230 patterns out of 823 by numeric redundancy
found 593 patterns from 352 trees for batch_idx 15
start score sort for batch_idx 15 (593) patterns
start merge rule for batch_idx 15 (593) patterns
[('hue-mean', True, 0.19106), ('rawred-mean', True, 0.00616), ('region-centroid-row', True, 0.56598), ('region-centroid-col', True, 0.48102)]
0.839622641509434 0.053003558825451456 0.12245420003529157 0.21730747419757315
merge complete for batch_idx 15 (593) patterns
start get explainer for batch_idx 15
as_chirps for batch_idx 16
start mining for batch_idx 16 with support = 0.1
reduced 23 patterns out of 638 by numeric redundancy
found 615 patterns from 500 trees for batch_idx 16
start score sort for batch_idx 16 (615) patterns
start merge rule for batch_idx 16 (615) patterns
[('exgreen-mean', True, 0.47352), ('hue-mean', False, 0.20033), ('rawred-mean', True, 0.18124), ('exred-mean', False, 0.68687), ('vedge-mean', True, 0.09601)]
0.9686274509803922 0.133536386380942 0.1028974505313288

[('hue-mean', False, 0.4665788374841213)]
0.8590163934426229 0.15691124608150472 0.1829523047177848 0.27522255192878337
merge complete for batch_idx 29 (12) patterns
start get explainer for batch_idx 29
Working on CHIRPS for instance 30 of 462
as_chirps for batch_idx 30
start mining for batch_idx 30 with support = 0.1
found 12 patterns from 500 trees for batch_idx 30
start score sort for batch_idx 30 (12) patterns
start merge rule for batch_idx 30 (12) patterns
[('hue-mean', False, 0.4665788374841213)]
0.8590163934426229 0.15691124608150472 0.18295231206781243 0.27522255192878337
merge complete for batch_idx 30 (12) patterns
start get explainer for batch_idx 30
as_chirps for batch_idx 31
start mining for batch_idx 31 with support = 0.1
reduced 23 patterns out of 638 by numeric redundancy
found 615 patterns from 500 trees for batch_idx 31
start score sort for batch_idx 31 (615) patterns
start merge rule for batch_idx 31 (615) patterns
[('exgreen-mean', True, 0.47352), ('hue-mean', False

[('hue-mean', False, 0.4665788374841213)]
0.8590163934426229 0.15691124608150472 0.18295220781086272 0.27522255192878337
merge complete for batch_idx 44 (12) patterns
start get explainer for batch_idx 44
Working on CHIRPS for instance 45 of 462
as_chirps for batch_idx 45
start mining for batch_idx 45 with support = 0.1
reduced 1 patterns out of 42 by numeric redundancy
found 41 patterns from 500 trees for batch_idx 45
start score sort for batch_idx 45 (41) patterns
start merge rule for batch_idx 45 (41) patterns
[('hue-mean', True, 0.61909), ('region-centroid-row', False, 0.6098958477377892), ('rawgreen-mean', False, 0.13949)]
0.9706959706959707 0.14322848637234414 0.26450977125119984 0.3571750902527075
merge complete for batch_idx 45 (41) patterns
start get explainer for batch_idx 45
as_chirps for batch_idx 46
start mining for batch_idx 46 with support = 0.1
reduced 13 patterns out of 588 by numeric redundancy
found 575 patterns from 500 trees for batch_idx 46
start score sort for bat

reduced 308 patterns out of 660 by numeric redundancy
found 352 patterns from 500 trees for batch_idx 59
start score sort for batch_idx 59 (352) patterns
start merge rule for batch_idx 59 (352) patterns
[('hue-mean', True, 0.18133), ('rawred-mean', True, 0.00347), ('region-centroid-row', True, 0.50479)]
0.8604651162790697 0.04243021216807894 0.13766564602007525 0.22392256354733933
merge complete for batch_idx 59 (352) patterns
start get explainer for batch_idx 59
Working on CHIRPS for instance 60 of 462
as_chirps for batch_idx 60
start mining for batch_idx 60 with support = 0.1
reduced 1 patterns out of 42 by numeric redundancy
found 41 patterns from 500 trees for batch_idx 60
start score sort for batch_idx 60 (41) patterns
start merge rule for batch_idx 60 (41) patterns
[('hue-mean', True, 0.61909), ('region-centroid-row', False, 0.6098958477377892), ('rawgreen-mean', False, 0.13949)]
0.9706959706959707 0.14322848637234414 0.2645098366354013 0.3571750902527075
merge complete for batch

...

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/site-packages/sklearn/tree/_classes.py", line 1246, in fit
    X_idx_sorted=X_idx_sorted)
  File "/usr/local/lib/python3.7/site-packages/sklearn/tree/_classes.py", line 336, in fit
    criterion = CRITERIA_REG[self.criterion](self.n_outputs_,
KeyError: 'poisson'

  FitFailedWarning)
/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/li

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/site-packages/sklearn/tree/_classes.py", line 1246, in fit
    X_idx_sorted=X_idx_sorted)
  File "/usr/local/lib/python3.7/site-packages/sklearn/tree/_classes.py", line 336, in fit
    criterion = CRITERIA_REG[self.criterion](self.n_outputs_,
KeyError: 'poisson'

  FitFailedWarning)
/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/li

0.0 in: 2.6226043701171875e-05


/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/site-packages/sklearn/tree/_classes.py", line 1246, in fit
    X_idx_sorted=X_idx_sorted)
  File "/usr/local/lib/python3.7/site-packages/sklearn/tree/_classes.py", line 336, in fit
    criterion = CRITERIA_REG[self.criterion](self.n_outputs_,
KeyError: 'poisson'

  FitFailedWarning)
/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/li

21.65 in: 16744.973735570908
43.29 in: 14817.445171117783
64.94 in: 15377.759154558182
86.58 in: 14918.46858882904


In [21]:
from utilities.lionforests_utility import path_similarity
rule_variance = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}


folds = 0
test_size = []
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=7)

min_max_feature_values = {}
for i in range(len(feature_names)):
        min_max_feature_values[feature_names[i]] = [min(X_train[:, i]), max(X_train[:, i])] 
for name in ['gs', 'ls', 'an', 'lf', 'df', 'ch']:
    for k in range(len(total_results2[0][folds][-1][name])):
        r1 = total_results2[0][0][-1][name][k]
        r2 = total_results2[1][0][-1][name][k]
        r3 = total_results2[2][0][-1][name][k]
        #try:
        rule_variance[name].append((path_similarity(r1, r2, feature_names, min_max_feature_values)+
                                       path_similarity(r1, r3, feature_names, min_max_feature_values)+
                                       path_similarity(r2, r3, feature_names, min_max_feature_values))/3)

In [22]:
f_full_coverage = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
f_rule_length = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
f_f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
f_f_time = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
for b in range(3):
    full_coverage = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
    rule_length = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
    f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
    f_time = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
    k = 0
    for i in total_results2[b]:
        for name, method in i[0].items():
            full_coverage[name].append(i[1][name]/test_size_2[b][k])
            rule_length[name].append(i[2][name]/test_size_2[b][k])
            f_precision[name].append(np.array(i[3][name]).mean())
            f_time[name].append(np.array(i[4][name]).mean())
        k = + 1
    for name, method in i[0].items():
        f_full_coverage[name].append(np.array(full_coverage[name]).mean())
        f_rule_length[name].append(np.array(rule_length[name]).mean())
        f_f_precision[name].append(np.array(f_precision[name]).mean())
        f_f_time[name].append(np.array(f_time[name]).mean())
for name, method in i[0].items():
    print(name,  '| %5.4f  %5.3f | %5.4f %5.3f | %5.4f  %5.3f | %5.4f  %5.3f | %5.4f  %5.3f' 
          % (np.array(f_full_coverage[name]).mean(),np.array(f_full_coverage[name]).std(),
             np.array(f_rule_length[name]).mean(),np.array(f_rule_length[name]).std(),
             np.array(f_f_precision[name]).mean(),np.array(f_f_precision[name]).std(),
             np.array(f_f_time[name]).mean(),np.array(f_f_time[name]).std(),
             np.array(rule_variance[name]).mean(),np.array(rule_variance[name]).std()))

gs | 0.0998  0.005 | 3.8874 0.064 | 0.9831  0.004 | 0.0004  0.000 | 0.7972  0.123
ls | 0.4269  0.001 | 1.7121 0.058 | 0.4422  0.007 | 2.3053  0.037 | 0.8811  0.295
an | 0.0586  0.002 | 5.8456 0.128 | 0.1446  0.002 | 142.7385  7.876 | 0.7378  0.346
lf | 0.0776  0.001 | 8.5065 0.005 | 1.0000  0.000 | 1.3310  0.019 | 0.9826  0.036
df | 0.9928  0.002 | 9.1003 0.261 | 0.6407  0.058 | 0.6497  0.020 | 0.7387  0.046
ch | 0.1394  0.001 | 3.7251 0.038 | 0.8818  0.003 | 1.1984  0.006 | 0.9437  0.048


In [23]:
import csv  

with open('segment.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    for name, method in i[0].items():
        writer.writerow([name, np.array(f_full_coverage[name]).mean(),np.array(f_full_coverage[name]).std(),
             np.array(f_rule_length[name]).mean(),np.array(f_rule_length[name]).std(),
             np.array(f_f_precision[name]).mean(),np.array(f_f_precision[name]).std(),
             np.array(f_f_time[name]).mean(),np.array(f_f_time[name]).std(),
             np.array(rule_variance[name]).mean(),np.array(rule_variance[name]).std()])